In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [2]:
width, height = 576, 576

new_width, new_height = 64, 64
left = (width - new_width)/2
top = (height - new_height)/2
right = (width + new_width)/2
bottom = (height + new_height)/2

(left, top, right, bottom)

(256.0, 256.0, 320.0, 320.0)

In [3]:
path =  os.path.join('idao_dataset', 'train', 'ER')
list_file_name = []
for x in os.listdir(path):
    list_file_name += [ os.path.join(path, x)] 
    
target_regres = []  
target_class = []
for name in list_file_name:
    names = os.path.split(name)[-1].split("_")
    idx = [i for i, v in enumerate(names) if v == "keV"][0]
    target_regres += [int(names[idx - 1])]
    target_class += [0]
    
path =  os.path.join('idao_dataset', 'train', 'NR')
for x in os.listdir(path):
    list_file_name += [ os.path.join(path, x)] 
    
ltr = len(target_regres)
for name in list_file_name[ltr:]:
    names = os.path.split(name)[-1].split("_")
    idx = [i for i, v in enumerate(names) if v == "keV"][0]
    target_regres += [int(names[idx - 1])]
    target_class += [1]

In [4]:
img_array_train = []
for path in tqdm(list_file_name):
    with Image.open(path) as img:
        img = img.crop((224.0, 224.0, 352.0, 352.0))
        img_array_train += [ np.array(img) ]

  0%|          | 0/13404 [00:00<?, ?it/s]

In [5]:
train_index, test_index = train_test_split(np.arange(len(img_array_train)), test_size  = 0.2 ,shuffle=True, random_state = 228)

In [6]:
from collections import Counter
Counter(np.array(target_regres)[test_index])
# Counter({1: 395, 30: 432, 3: 453, 10: 472, 20: 456, 6: 473})
# 

Counter({1: 395, 30: 432, 3: 453, 10: 472, 20: 456, 6: 473})

In [7]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Conv2D, MaxPool2D, BatchNormalization, LeakyReLU, AveragePooling2D
from keras.layers import GlobalMaxPooling1D, GlobalMaxPool1D, BatchNormalization, ReLU
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate, concatenate
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
import pandas as pd
import numpy as np
from keras.regularizers import L1L2
import tensorflow as tf
import re
from keras import regularizers
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras import backend as K 

K.clear_session()

In [8]:
target_class = np.array(target_class)

In [9]:
img_array_train = np.expand_dims(img_array_train, axis = -1)

In [10]:
target_regres_new = np.zeros((len(target_regres), 6))
dict_change = {1:0, 3:1, 6:2, 10:3,20:4,30:5}

In [11]:
for i, x in enumerate(target_regres):
    target_regres_new[i, dict_change[x]] = 1

In [12]:
# target_regres_new
# model_reg.summary()

In [13]:
target_regres = np.array(target_regres)

In [14]:
img_array_train = img_array_train / 255

In [15]:
import math
def step_decay(epoch):
    initial_lrate = 0.0005
    drop = 0.975
    epochs_drop = 1.0
    lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
#     print(lrate)
    return lrate
lrate = LearningRateScheduler(step_decay)

In [16]:
adam = Adam(learning_rate=0.001)
krl = regularizers.l2(0.00)
inputs = Input(shape=(img_array_train.shape[1],img_array_train.shape[2], 1),)
hui = Conv2D(20, kernel_size = 6)(inputs)
# hui = BatchNormalization(momentum=0.1, axis=-1)(hui)
# hui = ReLU()(hui)
# hui = Dropout(0.3)(hui)

mp = MaxPool2D(12, strides = (4, 4))(hui)
mp = BatchNormalization(momentum=0.05, axis=-1)(mp)
# mp = ReLU()(mp).
fl = Flatten()(mp)
fl = Dropout(0.7)(fl)
fl = Dense(100, activation = 'relu', kernel_regularizer=krl)(fl)
outputs = Dense(6, activation='softmax')(fl)


checkpoint_path = "reg_saved-model-{epoch:02d}.hdf5"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor = 'val_loss',
                                                 
                                                 save_best_only=False, save_weights_only=True, verbose=0)

model_reg = Model(inputs=inputs, outputs=outputs)
model_reg.compile(loss='categorical_crossentropy',optimizer=adam,  metrics=['accuracy'])

print(model_reg.summary())

history = model_reg.fit(img_array_train[train_index], target_regres_new[train_index], 
batch_size=150, epochs=50, verbose=1, shuffle  = True, callbacks=[cp_callback, lrate],
#                         class_weight = {0:1,1:2,2:3,3:5,4:7,5:10},
                        validation_batch_size = 150,
    validation_data=((img_array_train[test_index],  target_regres_new[test_index])) )
#                         , 
#                  sample_weight =  target_regres[train_index] ** (1/3) )

# del model
# del adam
# K.clear_session()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 123, 123, 20)      740       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 20)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 20)        80        
_________________________________________________________________
flatten (Flatten)            (None, 15680)             0         
_________________________________________________________________
dropout (Dropout)            (None, 15680)             0         
_________________________________________________________________
dense (Dense)                (None, 100)              

Epoch 49/50
72/72 [==============================] - 3s 36ms/step - loss: 0.0044 - accuracy: 0.9987 - val_loss: 0.0057 - val_accuracy: 0.9981
Epoch 50/50
72/72 [==============================] - 3s 36ms/step - loss: 0.0041 - accuracy: 0.9986 - val_loss: 0.0071 - val_accuracy: 0.9985
